In [1]:
import pickle
import pandas as pd


In [2]:
with open('spam_classifier_model.pkl', 'rb') as file:
    model1 = pickle.load(file)


with open('count_vectorizer.pkl', 'rb') as file:
    count = pickle.load(file)


C:\Users\boont\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\boont\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
def checkSpam(df):
    if df.empty or 'email_text' not in df.columns:
        raise ValueError("DataFrame is empty or does not contain 'email_text' column.")
    
    email_text = df.iloc[0]['email_text']
    email_vector = count.transform([email_text])
    prediction = model1.predict(email_vector.toarray())
    return prediction[0] == 1  
data = {
    'email_text': [
        "Subject: Greetings, My name is Kuan Tian"
    ]
}
df = pd.DataFrame(data)


is_spam = checkSpam(df)
print(is_spam)  

True


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from nltk.stem import WordNetLemmatizer

In [6]:
# Used to install wordnet lemmatizer
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find either /usr/share/nltk_data/corpora/wordnet.zip or /usr/share/nltk_data/corpora/wordnet.zip.zip.


In [7]:
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np
import pandas as pd
import nltk
import os
import string

from tqdm import tqdm


In [8]:
import time, sys
from IPython.display import clear_output
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [9]:
msg_useful_terms_without_digit = pd.read_csv("msg_useful_terms_without_digit.csv")
msg_useful_terms_without_digit.replace(to_replace = np.nan, value = "", inplace=True)

thread_term_weights = pd.read_csv("thread_term_weights.csv")
thread_term_weights.replace(to_replace = np.nan, value = "", inplace=True)

thread_weights = pd.read_csv("thread_weights.csv")
thread_weights.replace(to_replace = np.nan, value = "", inplace=True)

df = pd.read_csv("testset.csv")
df.replace(to_replace = np.nan, value = "", inplace=True)

In [10]:
df.shape

(108, 4)

In [11]:
df

,subject,content,email_text,label
0,Free entry in 2 a wkly comp to win FA Cup fina...,Text FA to 87121 to receive entry question(std...,Free entry in 2 a wkly comp to win FA Cup fina...,0
1,WINNER!!,As a valued network customer you have been sel...,WINNER!! As a valued network customer you have...,0
2,FreeMsg,Hey there darling it's been 3 week's now and n...,FreeMsg Hey there darling it's been 3 week's n...,0
3,Had your mobile 11 months or more?,U R entitled to Update to the latest colour mo...,Had your mobile 11 months or more? U R entitle...,0
4,SIX chances to win CASH!,"From 100 to 20,000 pounds txt> CSH11 and send ...","SIX chances to win CASH! From 100 to 20,000 po...",0
...,...,...,...,...
103,oasis transport imbalance update,I have attached the draft policy for your revi...,oasis transport imbalance update I have attach...,2
104,new book theory financial risk update,Please find the attached project timeline and ...,new book theory financial risk update Please f...,2
105,start date hourahead hour update,The audit report is attached. Please review an...,start date hourahead hour update The audit rep...,2
106,call note update,I have attached the meeting agenda for your re...,call note update I have attached the meeting a...,2


In [12]:
def get_weights(search_term, weight_df, term=True):
    if (len(search_term)>0):
        if term:
            term_match = False
            for search_item in search_term:
                match = weight_df["term"] == search_item
                term_match = term_match | match
        else:
            term_match = weight_df.subject.str.contains(search_term, regex=False)
        
        match_weights = weight_df.weight[term_match]
        if len(match_weights)<1:
            return 1
        else:
            return match_weights.mean()
    else:
        return 1

In [13]:
def rank_message(msg):        
    # Then, from thread activity
    is_thread = len(msg.subject.split('re ')) > 1
    if is_thread:
        subject = msg.subject.split('re ')[1]
        msg_thread_activity_wt = get_weights(subject, thread_weights, term=False)
    else:
        msg_thread_activity_wt = 1
    
    try:
        vec = CountVectorizer()
        sub_vec = vec.fit_transform([msg['subject']])
        msg_thread_terms = vec.get_feature_names_out()
        msg_thread_term_wt = get_weights(msg_thread_terms, thread_term_weights)
    except:
        # Some subjects from the test set result in empty vocabulary
        msg_thread_term_wt = 1
    
    try:
        vec = CountVectorizer()
        msg_vec = vec.fit_transform([msg['content']])
        msg_terms = vec.get_feature_names_out()
        msg_terms_wt = get_weights(msg_terms, msg_useful_terms_without_digit)
    except:
        # Some subjects from the test set result in empty vocabulary
        msg_terms_wt = 1
    
    # Calculating Rank
    rank = float(msg_thread_activity_wt) * float(msg_thread_term_wt) * float(msg_terms_wt)
    
    return rank, msg.subject

In [14]:
def remove_unwanted_words(x):
    res = []
    for i, word in enumerate(x):
        if word.isdigit():
            continue
        if word not in unwanted_words:
            res.append(word)
            continue
        if i == 0 and (word in ["re", "fw"]):
            res.append(word)
            continue
    return res

unwanted_words = [] + list(nltk.corpus.stopwords.words("english")) + list(string.punctuation)

In [15]:
def lemmatization(x):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in x]

In [16]:
df["subject"] = df["subject"].apply(lambda x:x.lower())
df["content"] = df["content"].apply(lambda x:x.lower())
df["subject"] = df["subject"].apply(nltk.word_tokenize)
df["content"] = df["content"].apply(nltk.word_tokenize)
# stop word, punctuation, digit removal
df["subject"] = df["subject"].apply(remove_unwanted_words)
df["content"] = df["content"].apply(remove_unwanted_words)
# lemmatization
df["subject"] = df["subject"].apply(lemmatization)
df["subject"] = df["subject"].apply(" ".join)
df["content"] = df["content"].apply(lemmatization)
df["content"] = df["content"].apply(" ".join)

In [17]:
df

,subject,content,email_text,label
0,free entry wkly comp win fa cup final tkts 21s...,text fa receive entry question std txt rate c ...,Free entry in 2 a wkly comp to win FA Cup fina...,0
1,winner,valued network customer selected receivea å£90...,WINNER!! As a valued network customer you have...,0
2,freemsg,hey darling 's week 's word back 'd like fun s...,FreeMsg Hey there darling it's been 3 week's n...,0
3,mobile month,u r entitled update latest colour mobile camer...,Had your mobile 11 months or more? U R entitle...,0
4,six chance win cash,"20,000 pound txt csh11 send 87575. cost 150p/d...","SIX chances to win CASH! From 100 to 20,000 po...",0
...,...,...,...,...
103,oasis transport imbalance update,attached draft policy review comment please pr...,oasis transport imbalance update I have attach...,2
104,new book theory financial risk update,please find attached project timeline mileston...,new book theory financial risk update Please f...,2
105,start date hourahead hour update,audit report attached please review prepare fo...,start date hourahead hour update The audit rep...,2
106,call note update,attached meeting agenda reference please revie...,call note update I have attached the meeting a...,2


In [18]:
correct = {
    0: [0,0], # number of correctly predicted, total number of spam dataset
    1: [0,0],
    2: [0,0],
}

for i in range(df.shape[0]):
    correct[df.iloc[i, :].label][1] += 1
    spam = checkSpam(pd.DataFrame(df.iloc[[i]]))
    result = None
    res = None
    if spam:
        result = 0
    
    if not spam:
        # tokenization
        res = rank_message(df.iloc[i, :])
        result = 2 if (res[0] > 9.009694517030455) else 1
    if result == df.iloc[i, :].label: correct[df.iloc[i, :].label][0] += 1
    print(f"Test {i+1}\nPredict: {result}\nActual: {(df.iloc[i, :].label)}")
    if res is not None: print(f"Rank {res[0]}")
    print()

print("Spam Dataset")
print(f"Accuracy : {correct[0][0]}/{correct[0][1]} = {correct[0][0] / correct[0][1]}")
print()
print("Not Important Dataset")
print(f"Accuracy : {correct[1][0]}/{correct[1][1]} = {correct[1][0] / correct[1][1]}")
print()
print("Important Dataset")
print(f"Accuracy : {correct[2][0]}/{correct[2][1]} = {correct[2][0] / correct[2][1]}")
print()
print(f"Total Accuracy : {correct[0][0] + correct[1][0] + correct[2][0]}/{correct[0][1] + correct[1][1] + correct[2][1]} = {(correct[0][0] + correct[1][0] + correct[2][0]) / (correct[0][1] + correct[1][1] + correct[2][1])}")

Test 1
Predict: 0
Actual: 0

Test 2
Predict: 0
Actual: 0

Test 3
Predict: 0
Actual: 0

Test 4
Predict: 0
Actual: 0

Test 5
Predict: 0
Actual: 0

Test 6
Predict: 0
Actual: 0

Test 7
Predict: 0
Actual: 0

Test 8
Predict: 0
Actual: 0

Test 9
Predict: 0
Actual: 0

Test 10
Predict: 0
Actual: 0

Test 11
Predict: 0
Actual: 0

Test 12
Predict: 0
Actual: 0

Test 13
Predict: 0
Actual: 0

Test 14
Predict: 0
Actual: 0

Test 15
Predict: 2
Actual: 0
Rank 32.522216553466734

Test 16
Predict: 0
Actual: 0

Test 17
Predict: 0
Actual: 0

Test 18
Predict: 0
Actual: 0

Test 19
Predict: 0
Actual: 0

Test 20
Predict: 0
Actual: 0

Test 21
Predict: 0
Actual: 0

Test 22
Predict: 0
Actual: 0

Test 23
Predict: 0
Actual: 0

Test 24
Predict: 0
Actual: 0

Test 25
Predict: 0
Actual: 0

Test 26
Predict: 0
Actual: 0

Test 27
Predict: 2
Actual: 0
Rank 36.71697123858817

Test 28
Predict: 0
Actual: 0

Test 29
Predict: 0
Actual: 0

Test 30
Predict: 0
Actual: 0

Test 31
Predict: 0
Actual: 0

Test 32
Predict: 0
Actual: 0

Te